In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import glob
import time 
import numpy as np
from scipy import optimize
import numba as nb
from mwc_class import getMVWs
from mwc_functions import *
from optimization_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
start_time=time.time()
italy_data = getMVWs('italy.csv', name='italy', save_results=True,verify_mwcs=True)
prelims = italy_data.preliminaries()
country_MIW = italy_data.minimal_voting_weights_pipeline()
end_time=time.time()
duration_Verify=start_time-end_time
print(duration_Verify)


{'1987': True, '1992': True, '1994': True, '1996': True, '2001': True, '2006': True, '2008': True, '2013': True, '2018': True}
{'1987': {}, '1992': {}, '1994': {}, '1996': {}, '2001': {}, '2006': {}, '2008': {}, '2013': {}, '2018': {}}
-25.007450819015503


# Loop for all countries:

In [74]:
def country_name_from_file(file_path):
    #gets the name of the country from the currently 'inspected' file
    #helper function for next cell
    file = os.path.basename(file_path)
    country_name, _ = os.path.splitext(file)
    return country_name.capitalize()



In [95]:
folder = 'data/'
cases_to_be_looked_at = {}
start_time = time.time()

for csv_file in glob.glob(os.path.join(folder, '*.csv')):
    country_name = country_name_from_file(csv_file)
    country_start_time = time.time()
    country_data = getMVWs(f'{country_name}.csv', name=country_name, save_results=True,verify_mwcs=True)
    
    prelims = country_data.preliminaries()
    country_MIW = country_data.minimal_voting_weights_pipeline()
    country_end_time = time.time()
    country_duration = country_end_time-country_start_time
    print(f'{country_name} took {country_duration} seconds')
    questionable_years=[]
    for year,dict in country_data.errors.items(): 
        if dict: 
            questionable_years.append(year)
    if questionable_years: 
        cases_to_be_looked_at[country_name]=questionable_years        
end_time = time.time()
total_time = end_time - start_time
print(f"Total time: {total_time:.2f} seconds")

Australia took 0.1648576259613037 seconds
Austria took 0.1146705150604248 seconds
Belgium took 5.538304328918457 seconds
Bulgaria took 0.14963912963867188 seconds
Canada took 0.09987616539001465 seconds
Croatia took 0.3959934711456299 seconds
Cyprus took 0.12132954597473145 seconds
Czech-republic took 0.2834126949310303 seconds
Denmark took 0.600499153137207 seconds
Estonia took 0.13360166549682617 seconds
Finland took 0.7111649513244629 seconds
France took 5.406847715377808 seconds
Germany took 0.1611166000366211 seconds
Greece took 0.09440803527832031 seconds
Hungary took 0.1150047779083252 seconds
Iceland took 0.20254302024841309 seconds
Ireland took 0.3498506546020508 seconds
Israel took 9.709914922714233 seconds
Italy took 25.307467222213745 seconds
Japan took 0.5602259635925293 seconds
Latvia took 0.1989762783050537 seconds
Lithuania took 2.4638328552246094 seconds
Luxembourg took 0.12251067161560059 seconds
Malta took 0.05018448829650879 seconds
Netherlands took 2.74257230758667

Anything below was originally to create the now exisitng pipeline and is not needed but documents the development process. 

# Where is waldo section

## Belgium 

In [11]:
country_data = getMVWs('italy.csv', name='italy', save_results=True,verify_mwcs=True)
prelims = country_data.preliminaries()
country_MIW = country_data.minimal_voting_weights_pipeline()

In [ ]:
spain = getMVWs('belgium.csv',name='Belgium',save_results=False,verify_mwcs=False)

In [ ]:
belgian_prelims = spain.preliminaries()

In [ ]:
transformed_dataframe = belgian_prelims.get("transformed_dataframe")
parties_in_year = belgian_prelims.get("parties_in_year")
n_in_year = belgian_prelims.get("n_in_year")
totalseats_in_year = belgian_prelims.get("totalseats_in_year")
coalition_dict = belgian_prelims.get("coalition_dict")
winning_coal_dict = belgian_prelims.get("winning_coal_dict")
minimal_winning_coalitions = belgian_prelims.get("minimal_winning_coalitions")
maximal_losing_coalitions = belgian_prelims.get("maximal_losing_coalitions")
unique_tying_coalitions = belgian_prelims.get("unique_tying_coalitions")

In [ ]:
start_time=time.time()
belgian_miw = spain.minimal_voting_weights_pipeline()
end_time=time.time()
runtime_noverify=end_time-start_time
print(runtime_noverify)

In [ ]:
spain.saveresults=True
spain.save_pipeline()

results look fishy

In [ ]:
austria_Data = getMVWs('austria.csv', name='austria', save_results=True,verify_mwcs=True)
    
prelims = austria_Data.preliminaries()
country_MIW = austria_Data.minimal_voting_weights_pipeline()


but seem to work for smaller parliaments

lets check belgium again, with adding unique tying constraints, first: baseline 

In [ ]:
relevant_coals=combine_dicts(spain.minimal_winning_coalitions,spain.maximal_losing_coalitions)
df_for_belgium = create_all_year_dfs(relevant_coals,parties_in_year)
belgian_mwc_constraints = generate_constraints_df(df_for_belgium['2014'])
belgian_lin_cons= get_lin_cons(belgian_mwc_constraints)
beligan_mvw=get_min_vote_weights('2014',n_in_year,belgian_lin_cons)
print(beligan_mvw.x)
print(sum(beligan_mvw.x))

trying to add more structure to belgian data

In [ ]:
def append_tying_coals_to_cons(year,constraints_df, unique_tying,parties_in_year):
    ##takes in constraints_df and adds unique-tying constraints to it 
    constraints_copy=constraints_df.copy()
    constraint_index = len(constraints_copy)  # make sure to add to the bottom 
    
    for (yr, (coalition, complementary_coal)) in unique_tying.keys():
        if yr==year: #make sure to only add contraints of the correct year
            constraint_row = {f'w_{party}': 0 for party in parties_in_year[year]}  # Initialize all party weights to 0 #create a dict for every row - first set keys to column names and attribute all of them value 0 
            for party in coalition.split('+'): #update value dapending on unique-tying dict 
                constraint_row[f'w_{party}'] = 1
            for party in complementary_coal.split('+'):
                constraint_row[f'w_{party}'] = -1
            # Add to df
            constraints_copy.loc[constraint_index] = constraint_row
            constraint_index += 1

    return constraints_copy

In [ ]:
new_constraints = append_tying_coals_to_cons('2014',belgian_mwc_constraints,unique_tying_coalitions,parties_in_year)

need to account for differnet constraint form of unique tying coals "(tying_coal)-(complement)"=0

In [ ]:
def get_new_lin_cons(year,constraints_df,unique_tying_coalitions):
    # now also incorporates the fact the the unique tying coals have to be exactly 0    
    A = constraints_df.to_numpy()
    lbnd = np.zeros(len(constraints_df)) # non-negativity constraints 
    lbnd[constraints_df.shape[1]:] = 1 #set all remaining lower bounds to 1
    upbnd = np.full(len(constraints_df), np.inf) #no upper bound for most constraints 
    #get # of unique tying coals 
    nr_of_ties = sum(1 for key in unique_tying_coalitions.keys() if key[0] == year) #count how many keys there are for the year 
    upbnd[-nr_of_ties:]=0 
    lbnd[-nr_of_ties:]=0

    lin_cons = LinearConstraint(A, lbnd, upbnd)

    return lin_cons

In [ ]:
new_lin_cons = get_new_lin_cons('2014',new_constraints,unique_tying_coalitions)

In [ ]:
get_min_vote_weights('2014',n_in_year,new_lin_cons)

In [ ]:
print(beligan_mvw.x)

adding more constraints does not change the result 

double belgian parliament and test for result 

In [ ]:
lithuania = read_csv_to_dataframe('belgium.csv')
df = transform_and_sort_dataframe(lithuania)
df['# of Seats']=df['# of Seats']*5
parties_in_year, totalseats_in_year, n_in_year= variables_by_year(df)
coalition_dict = coalition_combinatorics_generator(df,parties_in_year)

pipeline manual

In [ ]:
begin = time.time()
winning_coal_dict=win_coals(coalition_dict, totalseats_in_year)
min_win_coal_dict=min_winning_coals(winning_coal_dict)
maximal_losing_dict=max_loosing_coals(winning_coal_dict, parties_in_year)
combined_dict=combine_dicts(min_win_coal_dict, maximal_losing_dict)
all_year_dfs=create_all_year_dfs(combined_dict, parties_in_year)
constraints_df=generate_constraints_df(all_year_dfs['2014'])
lin_con=get_lin_cons(constraints_df)
min_weights=get_min_vote_weights('2014',n_in_year,lin_con)
end = time.time()
duration = end-begin 


In [ ]:
results=mvw_to_parties('2014',min_weights,parties_in_year)
(results)

also, same result, will believe it to be correct for now 

## Spain

spain 2019 - n = 18 

In [3]:

spain = getMVWs('spain.csv',name='Spain',save_results=False,verify_mwcs=False)
spain.preliminaries()
spain.minimal_voting_weights_pipeline()


{'Dataframes': {'1993':                                                     CC - Canary Coalition  (Coalición Canaria, CC)  \
  CiU - Convergence and Union (Convergencia i Uni...                                               0   
  IU - United Left (Izquierda Unida, IU)+PSOE - S...                                               0   
  PP - Popular Party (Partido Popular, PP)+PSOE -...                                               0   
  CiU - Convergence and Union (Convergencia i Uni...                                               0   
  CC - Canary Coalition  (Coalición Canaria, CC)+...                                               1   
  CC - Canary Coalition  (Coalición Canaria, CC)+...                                               1   
  CC - Canary Coalition  (Coalición Canaria, CC)+...                                               1   
  CC - Canary Coalition  (Coalición Canaria, CC)+...                                               1   
  
                                       

kinda cracked, ngl :D

## israel

In [ ]:
start_time = time.time()

israel_data = getMVWs('israel.csv', name='Israel', save_results=False,find_errors=True)
prelims = israel_data.preliminaries()
prelim_time = israel_data.time - start_time
print(prelim_time)
country_MIW = israel_data.minimal_voting_weights_pipeline()
pipeline_time = israel_data.time - prelim_time
print(pipeline_time)
end_time = time.time()
total_time = end_time - start_time
print(f"Total time: {total_time:.2f} seconds") 

investigate israel 1999:

In [ ]:
israel_data = getMVWs('israel.csv',name='israel',save_results=False,verify_mwcs=False,find_errors=True)

In [ ]:
israel_prelims = israel_data.preliminaries()

In [ ]:
transformed_dataframe = israel_prelims.get("transformed_dataframe")
parties_in_year = israel_prelims.get("parties_in_year")
n_in_year = israel_prelims.get("n_in_year")
totalseats_in_year = israel_prelims.get("totalseats_in_year")
coalition_dict = israel_prelims.get("coalition_dict")
winning_coal_dict = israel_prelims.get("winning_coal_dict")
minimal_winning_coalitions = israel_prelims.get("minimal_winning_coalitions")
maximal_losing_coalitions = israel_prelims.get("maximal_losing_coalitions")
unique_tying_coalitions = israel_prelims.get("unique_tying_coalitions")

In [ ]:
begin = time.time()
combined_dict=combine_dicts(minimal_winning_coalitions, maximal_losing_coalitions)
all_year_dfs= create_all_year_dfs(combined_dict,parties_in_year)
end = time.time()
duration = end-begin 
print(duration)


In [ ]:

def constraint_generation_with_timer(df):


    parties = df.columns[:-1] 
    constraints_df = pd.DataFrame(columns=parties) #create new empty df with weights as columns  
    constraint_index = 0 # to allow .loc[] instead of .append, basically counts constraints  

    winning_coalitions = df[df['Winning'] == 1] # Split original DataFrame into winning and losing coalitions
    losing_coalitions = df[df['Winning'] == 0]

    # non-negativity constraints
    for party in parties:
        non_negative_constraint = {p: 1 if p == party else 0 for p in parties} # puts a 1 where respective p is equal to the party and 0 else 
        constraints_df.loc[constraint_index] = non_negative_constraint #adds rows for every constraint
        constraint_index += 1 #update index counter

    start_time = time.time()
    last_stamp = constraint_index 
    # w(S)>w(R) constraints
    for _, winning_row in winning_coalitions.iterrows(): #outerloop with the winning coals
        for _, losing_row in losing_coalitions.iterrows(): #inner loop with the losing coals
            
            # Create a dict for the constraint (party,value) where value \in {-1,0,1}
            constraint_row = {}
            for party in parties: #calculate difference between the winning and the losing coal
                constraint_row[party] = winning_row[party] - losing_row[party] #checks the value of the party in the winning coal (either 1 if present or 0 if not) and the losing coal. Example S={CDU,SPD,B90}, R={SPD,B90,FDP} then CDU=1-0, SPD=1-1, B90=1-1, FDP=0-1. Resulting constraint is w_cdu-w_fdp>0 

            # Add the constraint to the DataFrame
            constraints_df.loc[constraint_index] = constraint_row
            constraint_index += 1
            if constraint_index % 1000 == 0:
                    current_time = time.time()
                    print(f"Took {current_time - start_time:.2f} seconds to write constraints {last_stamp} to {constraint_index}")
                    start_time = current_time 
                    last_stamp = constraint_index
    # Renaming 
    constraints_df.columns = [f'w_{party}' for party in parties]

    return constraints_df



In [ ]:
df_1999=all_year_dfs['1999']

In [ ]:
begin = time.time()
constraint_generation_with_timer(df_1999)
end = time.time()

duration = end-begin 
print(duration)

need a more efficient constraints generator: 

In [ ]:
def generate_eff_cons(df):
    
    parties = df.columns[:-1].tolist()
    n = len(parties) 

    
    winning_coalitions = df[df['Winning'] == 1] 
    losing_coalitions = df[df['Winning'] == 0]
    
    winning_array = winning_coalitions.drop(columns=['Winning']).to_numpy()
    losing_array = losing_coalitions.drop(columns=['Winning']).to_numpy()
    
    n_constraints = len(winning_array) * len(losing_array) + n
    constraints_array = np.zeros((n_constraints, n),dtype='int') #store constraints in an array, as its much faster than a df, initialize 0´s everywhere
    
    # Non-negativity constraints
    for i, party in enumerate(parties): #enum creates (1,party_1),(2,party_2),... elements thus needs to call i,party even if party is never accessed 
        constraints_array[i, i] = 1 # write a 1 where party=party 
    
    # w(S) > w(R) constraints
    constraint_index = n  # Start after non-negativity cons
    for win_row in winning_array:
        for lose_row in losing_array:
            # #calculate difference between the winning and the losing coal
            constraints_array[constraint_index] = win_row - lose_row #checks the value of the party in the winning coal (either 1 if present or 0 if not) and the losing coal. Example S={CDU,SPD,B90}, R={SPD,B90,FDP} then CDU=1-0, SPD=1-1, B90=1-1, FDP=0-1. Resulting constraint is w_cdu-w_fdp>0 
            constraint_index += 1
            
            
    # array to original df 
    constraints_df = pd.DataFrame(constraints_array, columns=[f'w_{party}' for party in parties])
    
    return constraints_df


In [ ]:
begin = time.time()
cons_1999= generate_eff_cons(df_1999)
end = time.time()

check whether new dfs are correct: 

In [ ]:
old_data_frames={}
for year, df in austria_Data.all_constraints.items():
    old_data_frames[year] = df  
new_data_frames = {}
for year,df in austria_Data.all_dfs.items(): 
    new_data_frames[year]=generate_eff_cons(austria_Data.all_dfs[year]) 
    test_dict ={}
for year,df in old_data_frames.items():
    is_identical = new_data_frames[year].equals(old_data_frames[year].convert_dtypes())
    test_dict[year]=is_identical
print(test_dict)
    

In [ ]:
#quick and dirty check to see whether df´s are the same: 

def find_differences(df1, df2):
    if set(df1.columns) != set(df2.columns):
        return "different columns."
    if not df1.index.equals(df2.index):
        return "different indices."
    diff_mask = (df1 != df2).any(axis=1)
    diff_indices = df1.index[diff_mask]

    return diff_indices

diff_indices = find_differences(old_data_frames['1994'], new_data_frames['1994'])
print(diff_indices)

find the difference aka where´s waldo: 

In [ ]:
print(old_data_frames['1994'].dtypes)
print(new_data_frames['1994'].dtypes)

there is waldo....

In [ ]:
test_dict ={}
for year,df in old_data_frames.items():
    old_data_frame =old_data_frames[year].apply(lambda col: col.astype('int32'))
    is_identical = new_data_frames[year].equals(old_data_frame)
    test_dict[year]=is_identical
print(test_dict)

q.e.d.

test new pipeline with "challenging" isrealian data: 

In [3]:
start_time=time.time()
test_Data = getMVWs('israel.csv', name='israel', save_results=False,verify_mwcs=False)
prelims = test_Data.preliminaries()
country_MIW = test_Data.minimal_voting_weights_pipeline()
end_time=time.time()
duration_no_Verify=start_time-end_time
print(duration_no_Verify)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hiwis\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Hiwis\AppData\Local\Temp\ipykernel_13904\2006158271.py", line 4, in <module>
    country_MIW = test_Data.minimal_voting_weights_pipeline()
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Hiwis\Documents\Sebastian Masterthesis\Git repo\masterthesis\mwc_class.py", line 177, in minimal_voting_weights_pipeline
    self.All_the_optimal_seats()
  File "c:\Users\Hiwis\Documents\Sebastian Masterthesis\Git repo\masterthesis\mwc_class.py", line 75, in All_the_optimal_seats
    self.test_bool,self.optimal_seats,self.wrong_coals = get_all_optimized_seats(self.all_min_weights,self.parties_in_year, self.all_relevant_coals,self.verify)
                                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

this takes about 6.9 seconds 

In [5]:
start_time=time.time()
israelian_Data = getMVWs('israel.csv', name='israel', save_results=False,verify_mwcs=True)
prelims = israelian_Data.preliminaries()
country_MIW = israelian_Data.minimal_voting_weights_pipeline()
end_time=time.time()
duration_Verify=start_time-end_time
print(duration_Verify)

{'1992': True, '1996': True, '1999': False, '2003': True, '2006': True, '2009': True, '2013': True, '2015': True, '2019': False, '2020': True}
{'1992': {}, '1996': {}, '1999': {('1999', 'CP +Lab+Mer+Sha'): (0, 1), ('1999', 'Isr+Lab+Lik+Mer'): (0, 1), ('1999', 'Lab+Lik+Mer+NRP'): (0, 1), ('1999', 'Lab+Lik+Mer+Raa'): (0, 1), ('1999', 'Lab+Lik+Mer+YH '): (0, 1), ('1999', 'Lab+Mer+Sha+Shi'): (0, 1), ('1999', 'Lab+Mer+Sha+YBA'): (0, 1), ('1999', 'AE +Bal+Lab+Lik+Mer'): (0, 1), ('1999', 'AE +CP +Lab+Lik+Shi'): (0, 1), ('1999', 'AE +CP +Lab+Lik+YBA'): (0, 1), ('1999', 'AE +Had+Lab+Lik+Mer'): (0, 1), ('1999', 'AE +Isr+Lab+Mer+Sha'): (0, 1), ('1999', 'AE +Lab+Lik+Shi+YBA'): (0, 1), ('1999', 'AE +Lab+Mer+NRP+Sha'): (0, 1), ('1999', 'AE +Lab+Mer+Raa+Sha'): (0, 1), ('1999', 'AE +Lab+Mer+Sha+YH '): (0, 1), ('1999', 'Bal+CP +Lab+Lik+Shi'): (0, 1), ('1999', 'Bal+CP +Lab+Lik+YBA'): (0, 1), ('1999', 'Bal+Had+Lab+Lik+Mer'): (0, 1), ('1999', 'Bal+Isr+Lab+Mer+Sha'): (0, 1), ('1999', 'Bal+Lab+Lik+Shi+YBA')

In [6]:
israelian_Data.errors['1999']

{('1999', 'CP +Lab+Mer+Sha'): (0, 1),
 ('1999', 'Isr+Lab+Lik+Mer'): (0, 1),
 ('1999', 'Lab+Lik+Mer+NRP'): (0, 1),
 ('1999', 'Lab+Lik+Mer+Raa'): (0, 1),
 ('1999', 'Lab+Lik+Mer+YH '): (0, 1),
 ('1999', 'Lab+Mer+Sha+Shi'): (0, 1),
 ('1999', 'Lab+Mer+Sha+YBA'): (0, 1),
 ('1999', 'AE +Bal+Lab+Lik+Mer'): (0, 1),
 ('1999', 'AE +CP +Lab+Lik+Shi'): (0, 1),
 ('1999', 'AE +CP +Lab+Lik+YBA'): (0, 1),
 ('1999', 'AE +Had+Lab+Lik+Mer'): (0, 1),
 ('1999', 'AE +Isr+Lab+Mer+Sha'): (0, 1),
 ('1999', 'AE +Lab+Lik+Shi+YBA'): (0, 1),
 ('1999', 'AE +Lab+Mer+NRP+Sha'): (0, 1),
 ('1999', 'AE +Lab+Mer+Raa+Sha'): (0, 1),
 ('1999', 'AE +Lab+Mer+Sha+YH '): (0, 1),
 ('1999', 'Bal+CP +Lab+Lik+Shi'): (0, 1),
 ('1999', 'Bal+CP +Lab+Lik+YBA'): (0, 1),
 ('1999', 'Bal+Had+Lab+Lik+Mer'): (0, 1),
 ('1999', 'Bal+Isr+Lab+Mer+Sha'): (0, 1),
 ('1999', 'Bal+Lab+Lik+Shi+YBA'): (0, 1),
 ('1999', 'Bal+Lab+Mer+NRP+Sha'): (0, 1),
 ('1999', 'Bal+Lab+Mer+Raa+Sha'): (0, 1),
 ('1999', 'Bal+Lab+Mer+Sha+YH '): (0, 1),
 ('1999', 'CP +Had+L

In [12]:
israelian_Data.optimal_seats['1999']

{'AE ': 2.0,
 'Bal': 2.0,
 'CP ': 6.0,
 'Had': 3.0,
 'Isr': 4.0,
 'Lab': 26.0,
 'Lik': 19.0,
 'Mer': 10.0,
 'NRP': 5.0,
 'Raa': 5.0,
 'Sha': 17.0,
 'Shi': 6.0,
 'YBA': 6.0,
 'YH ': 5.0}

In [11]:
sum(israelian_Data.optimal_seats['1999'].values())

116.0

## Lithuania

Whatever is wrong with Lithuania (2008): 

In [71]:
lithuania = read_csv_to_dataframe('lithuania.csv')
lithuania['Party'] = lithuania['Party'].str[:30]
df = transform_and_sort_dataframe(lithuania)
df=df[df['Year']=='2008']

parties_in_year, totalseats_in_year, n_in_year= variables_by_year(df)
coalition_dict = coalition_combinatorics_generator(df,parties_in_year)
begin = time.time()
winning_coal_dict=win_coals(coalition_dict, totalseats_in_year)
min_win_coal_dict=min_winning_coals(winning_coal_dict)
maximal_losing_dict=max_loosing_coals(winning_coal_dict, parties_in_year)
combined_dict=combine_dicts(min_win_coal_dict, maximal_losing_dict)
all_year_dfs=create_all_year_dfs(combined_dict, parties_in_year)
cons= generate_eff_cons(all_year_dfs['2008'])
lin_cons=get_lin_cons(cons)
results=get_min_vote_weights('2008',n_in_year,lin_cons)



this works, but: 

In [ ]:
lithuania = read_csv_to_dataframe('lithuania.csv')
#lithuania['Party'] = lithuania['Party'].str[:30]
df = transform_and_sort_dataframe(lithuania)
df=df[df['Year']=='2008']

parties_in_year, totalseats_in_year, n_in_year= variables_by_year(df)
coalition_dict = coalition_combinatorics_generator(df,parties_in_year)
begin = time.time()
winning_coal_dict=win_coals(coalition_dict, totalseats_in_year)
min_win_coal_dict=min_winning_coals(winning_coal_dict)
maximal_losing_dict=max_loosing_coals(winning_coal_dict, parties_in_year)
combined_dict=combine_dicts(min_win_coal_dict, maximal_losing_dict)
all_year_dfs=create_all_year_dfs(combined_dict, parties_in_year)
cons= generate_eff_cons(all_year_dfs['2008'])
lin_cons=get_lin_cons(cons)
results=get_min_vote_weights('2008',n_in_year,lin_cons)

this doesnt, for this reason im limiting length of party names to 30 chars, i suspect there is a problem with the combinatorics function, when the strings get too long 

In [72]:
results

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 141.00000000000003
              x: [ 1.000e+01  8.000e+00  3.000e+00  1.100e+01  2.500e+01
                   3.000e+00  1.000e+00  4.000e+00  1.600e+01  4.500e+01
                   1.500e+01]
 mip_node_count: 1
 mip_dual_bound: 141.00000000000003
        mip_gap: 0.0

In [62]:
cons

,w_KDPJ,w_LCS -,w_LLRA,w_LRLS,w_LSDP,w_LVZS,w_NSSL,w_Other,w_TPP -,w_TSLKD,w_TT -
0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3850,0,0,0,1,0,0,0,0,0,0,0
3851,0,0,1,0,0,0,0,0,0,0,0
3852,0,1,0,0,0,0,0,0,0,0,0
3853,1,0,0,0,-1,0,0,0,1,0,0


In [47]:
len(min_win_coal_dict)

63

In [48]:
len(maximal_losing_dict)

31

In [58]:
parties_in_year, totalseats_in_year, n_in_year

({'2008': ['KDPJ - Coalition of Labour Party\xa0+\xa0Youth (Koalicija Darbo partija\xa0+\xa0Jaunimas, KDPJ)',
   'LCS - Liberal and Centre Union (Liberalų ir centro sąjunga, LCS)',
   'LLRA - Electoral Action of Lithuanian Poles (Lietuvos lenkų rinkimų akcija, LLRA)',
   'LRLS - Liberal Movement of Lithuanian Republic (Lietuvos Respublikos Liberalų sąjūdis, LRLS)',
   'LSDP - Lithuanian Social Democratic Party (Lietuvos socialdemokratų partija, LSDP)',
   'LVZS - Lithuanian Peasant and Greens Union (Lietuvos valstiečių ir žaliųjų sąjunga,, LVZS), known until  as Union of Peasants’ and New Democracy Parties (VNDPS, Valstiečių ir Naujosios demokratijos partijų sąjungos), known from  until 2012 as Lithuanian Peasants People Union (LVLS, Lietuvos valstiečių liaudininkų sąjunga)',
   'NSSL - New Union (Social Liberals) (Naujoji sąjunga/ socialliberalai, NSSL)',
   'Other - Other and independent (Kiti ir nepriklausomas, Other)',
   "TPP - Nation's Resurrection Party (Tautos prisikėlimo parti

In [54]:
lithuania

,Party,Date,# of Seats,% of Seats,# of Votes,% of Votes
0,ABAPKUDL - Coalition of Algirdas Brazauskas an...,10-Oct-04,31,22.00,246852,20.65
1,ABSKK - Social Democratic Coalition of Algirda...,8-Oct-00,51,36.17,457294,31.08
2,"DK - Way of Courage (Drąsos kelias, DK)",14-Oct-12,7,5.00,109448,8.30
3,"DK - Way of Courage (Drąsos kelias, DK)",11-Oct-20,0,0.00,13337,1.10
4,"DP - Labour Party (Darbo partija, DP)",10-Oct-04,39,27.70,340035,28.44
...,...,...,...,...,...,...
77,TSLKD - Homeland Union/Lithuanian Christian De...,23-Oct-16,31,22.00,276275,22.60
78,TSLKD - Homeland Union/Lithuanian Christian De...,11-Oct-20,50,35.50,292124,24.90
79,"TT - Order and Justice (Tvarka ir teisingumas,...",12-Oct-08,15,10.64,156777,12.68
80,"TT - Order and Justice (Tvarka ir teisingumas,...",14-Oct-12,11,7.80,100120,7.60


## Netherlands

play the same game for netherlands: 

In [76]:
netherlands = read_csv_to_dataframe('netherlands.csv')
#netherlands['Party'] = netherlands['Party'].str[:30]
df = transform_and_sort_dataframe(netherlands)
#df=df[df['Year']=='2008']

parties_in_year, totalseats_in_year, n_in_year= variables_by_year(df)
coalition_dict = coalition_combinatorics_generator(df,parties_in_year)
winning_coal_dict=win_coals(coalition_dict, totalseats_in_year)
min_win_coal_dict=min_winning_coals(winning_coal_dict)
maximal_losing_dict=max_loosing_coals(winning_coal_dict, parties_in_year)
combined_dict=combine_dicts(min_win_coal_dict, maximal_losing_dict)
all_year_dfs=create_all_year_dfs(combined_dict, parties_in_year)
cons= get_all_constrains(all_year_dfs)
lin_cons=get_all_lin_cons(cons)
results=get_all_min_vote_weights(lin_cons,n_in_year)



TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable rint method

In [92]:
netherlands = read_csv_to_dataframe('netherlands.csv')

#lithuania['Party'] = lithuania['Party'].str[:30]
df = transform_and_sort_dataframe(netherlands)
#df=df[df['Year']=='2008']

parties_in_year, totalseats_in_year, n_in_year= variables_by_year(df)



In [93]:
totalseats_in_year

{'1994': 150,
 '1998': 150,
 '2002': 150,
 '2003': 150,
 '2006': 150,
 '2010': 150,
 '2012': 150,
 '2017': 150}

In [94]:
coalition_dict = coalition_combinatorics_generator(df,parties_in_year)
winning_coal_dict=win_coals(coalition_dict, totalseats_in_year)
min_win_coal_dict=min_winning_coals(winning_coal_dict)
maximal_losing_dict=max_loosing_coals(winning_coal_dict, parties_in_year)
combined_dict=combine_dicts(min_win_coal_dict, maximal_losing_dict)
all_year_dfs=create_all_year_dfs(combined_dict, parties_in_year)
cons= generate_eff_cons(all_year_dfs['2017'])
lin_cons=get_lin_cons(cons)
results=get_min_vote_weights('2017',n_in_year,lin_cons)
print(results)


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 149.99999999999991
              x: [ 4.000e+00  1.900e+01 ...  1.400e+01  3.300e+01]
 mip_node_count: 1
 mip_dual_bound: 149.99999999999991
        mip_gap: 0.0


In [90]:
len(coalition_dict)

18432

 ITS NOT THE LENGTH, ITS THE "+" IN PARTY NAMES

## cases to be looked at: 

In [96]:
cases_to_be_looked_at

{'Israel': ['1999', '2019']}

play around 

In [4]:
spain.minimal_winning_coalitions

{('1993',
  'CiU - Convergence and Union (Convergencia i Unió, CiU)+PSOE - Spanish Socialist Party (Partido Socialista Obrero Español, PSOE)'): 1,
 ('1993',
  'IU - United Left (Izquierda Unida, IU)+PSOE - Spanish Socialist Party (Partido Socialista Obrero Español, PSOE)'): 1,
 ('1993',
  'PP - Popular Party (Partido Popular, PP)+PSOE - Spanish Socialist Party (Partido Socialista Obrero Español, PSOE)'): 1,
 ('1993',
  'CiU - Convergence and Union (Convergencia i Unió, CiU)+IU - United Left (Izquierda Unida, IU)+PP - Popular Party (Partido Popular, PP)'): 1,
 ('1996',
  'IU - United Left (Izquierda Unida, IU)+PP - Popular Party (Partido Popular, PP)'): 1,
 ('1996',
  'PP - Popular Party (Partido Popular, PP)+PSOE - Spanish Socialist Party (Partido Socialista Obrero Español, PSOE)'): 1,
 ('1996',
  'CC - Canary Coalition  (Coalición Canaria, CC)+CiU - Convergence and Union (Convergencia i Unió, CiU)+PP - Popular Party (Partido Popular, PP)'): 1,
 ('1996',
  'CiU - Convergence and Union 